In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import re
import winsound
final_dataset = pd.DataFrame()

In [ ]:
#column_names = ["title", "domain", "abstract", "abstract_toc", "corr_author", "rec_day", "rec_mon", "rec_yr", "acc_day", "acc_mon", "acc_yr", "pub_day", "pub_mon", "pub_yr"]
paperinfo = pd.DataFrame()

for i in range(0, 40000):  #keep changing intervals (size of 40000) until you hit 220829
    try:
        file = ET.parse(f'D:/PhD Courses/Semester 4/Data Driven Design/Project Data/Article{i}.xml')
        root = file.getroot()
        pi = []
    
        if root.findall(".//subject/..[@subj-group-type='heading']") is not None:
            for heading in root.findall(".//subject/..[@subj-group-type='heading']"):
                if(str(heading[0].text)) == "Research Article":
                
                    if root.findall("./front/article-meta/title-group/article-title") is not None:
                        for title in root.findall("./front/article-meta/title-group/article-title"):
                            pi.append(title.text)
                            
                    else:
                        pi.append(str('no'))

                    #pull abstract
                    xmlstr = ET.tostring(root, encoding='utf8', method='xml')
                    xmlstr = xmlstr.decode('utf-8')
                    xmlstr = xmlstr.split()
                    xmlstr = " ".join(xmlstr)
                    start_abs = xmlstr.find("<abstract>") + len("<abstract>")
                    end_abs = xmlstr.find("</abstract>")
                    abstract = xmlstr[start_abs:end_abs]
                    abstract = re.sub('<.*?>', ' ', abstract)
                    pi.append(abstract)
                    
                    #pull discipline
                    if len(root.findall(".//subject/..[@subj-group-type='Discipline']")) == 0:
                        pi.append(np.NaN)
                         
                    else:
                        for subject in root.findall(".//subject/..[@subj-group-type='Discipline']"):
                            subj = subject[0].text
                            subj = subj.split("/")[0]
                            pi.append(subj)
                        

                    #Pull received date
                    xmlstr = re.sub(" ", "", xmlstr)
                    xmlstr = re.sub("\n", "", xmlstr)
                    if (xmlstr.find("received") != -1):
                        start_rec = xmlstr.find('<datedate-type="received">') + len('<datedate-type="received">')
                        end_rec = xmlstr.find("</date>")
                        rec_date = xmlstr[start_rec:end_rec]

                        rec_date = re.sub('<.*?>', '-', rec_date)
                        rec_date = re.sub('\n', '', rec_date)
                        rec_date = re.sub('--', '/', rec_date)
                        rec_date = re.sub('-', '', rec_date)
                        pi.append(rec_date)
    
                    else:
                        pi.append(str("NA"))

                    #Pull accepted date
                    if (xmlstr.find("accepted") != -1):
                        start_acc = xmlstr.find('<datedate-type="accepted">') + len('<datedate-type="accepted">')
                        end_acc = xmlstr.find("</history>")
                        acc_date = xmlstr[start_acc:end_acc]

                        acc_date = re.sub('<.*?>', '-', acc_date)
                        acc_date = re.sub('\n', '', acc_date)
                        acc_date = re.sub('--', '/', acc_date)
                        acc_date = re.sub('-', '', acc_date)
                        acc_date = acc_date[:-1]
                        pi.append(acc_date)
    
                    else:
                        pi.append(str("NA"))

                    #pull epub date
                    xmlstr = re.sub('</pub-date><pub-datepub-type="collection">.*?</year>', '', xmlstr)
                    if (xmlstr.find('pub-type="epub"') != -1):
                        start_epub = xmlstr.find('<pub-datepub-type="epub">') + len('<pub-datepub-type="epub">')
                        end_epub = xmlstr.find("</pub-date><volume>")
                        epub_date = xmlstr[start_epub:end_epub]

                        epub_date = re.sub('<.*?>', '-', epub_date)
                        epub_date = re.sub('\n', '', epub_date)
                        epub_date = re.sub('--', '/', epub_date)
                        epub_date = re.sub('-', '', epub_date)
                        pi.append(epub_date)
    
                    else:
                        pi.append(str("NA"))
                    
                    #pull author names
                    for author in root.findall(".//name/..[@contrib-type='author']"):
                        name = author[0][1].text
                        name = name.replace(".", " ")
                        name = re.sub(r"\b[a-zA-Z]\b", "", name)
                        pi.append(name)
         
                    pi = np.array(pi)
                    df = pd.DataFrame(pi.reshape(-1, len(pi)))
                    paperinfo = paperinfo.append(df)
                
                else:
                    i = i+1

    except:
        pass
    
winsound.Beep(480, 10000)

In [ ]:
paperinfo

In [ ]:
final_dataset = final_dataset.append(paperinfo)

In [ ]:
final_dataset

In [ ]:
final_dataset.to_csv(r'D:/PhD Courses/Semester 4/Data Driven Design/ArticleInfo.csv', index = False)